### PNS ELEC4 EIEL821 Spectral Analysis 

# Chapter 3: Parametric estimation - solutions

This Jupyter notebook aims at studying parametric techniques for spectral analysis, with particular focus on **AR modeling**.


## Model mismatch


**[HAY96, p. 441]** The performance of parametric techniques strongly depends on the fitness of the model. If the assumed model is inappropriate for the process under analysis, the parametric approach will lead to inaccurate or misleading spectral estimates. To illustrate this phenomenon, let us first consider a stochastic process consisting of **two sinusoids in additive noise**:

$$X(n) = 10\sin(0.4\pi n + \phi_1) + 5\sin(0.6\pi n + \phi_2) + \nu(n)$$

where $\nu(n)$ is a zero-mean unit-variance white noise and random phases $\phi_1$ and $\phi_2$ are distributed uniformly in $[0, 2\pi]$ rad.


* Generate 50 realizations of $N = 64$ samples of process $X(n)$. Plot one realization.


* Justify why an AR(4) model would be appropriate for this process. Determine the AR model coefficients by solving the Yule-Walker equations based on biased ACS estimates. Derive the AR spectral estimate (in dB) of each signal realization. Superimpose the 50 spectral realizations in the same figure, with $\omega$ in the interval $[0, \pi]$ rad/sample.


* Compute the average spectral estimate and plot it (in dB) in a different figure. Compare the average with the theoretical PSD of the process considered in this exercise. Superimpose (in dB) the variance of the spectral realizations. Compute the average variance over the whole frequency range.


* Repeat using the periodogram as spectral estimator. Compare.


Now assume that the process under analysis is actually governed by the **MA(2) model**:

$$X(n) = \varepsilon(n) - \varepsilon(n-2)$$

with theoretical PSD given by

$$S_X(\omega) = \left|1 - \mathrm{e}^{-\jmath 2\omega}\right|^2 = (1 - \mathrm{e}^{-\jmath 2\omega})(1 - \mathrm{e}^{\jmath 2\omega}) = 2 - 2\cos(2\omega).$$


* Repeat the above exercise for 50 realizations of this MA(2) random process. Compare the theoretical PSD with the AR(4) and periodogram estimates. Conclude.


*Hint:* For the spectral representations, sample the frequency axis in $N_\mathrm{FFT} = 1024$ equally-spaced points in the interval $[0, 2\pi]$ rad/sample.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import toeplitz

pi = np.pi
fft = np.fft.fft


# ======================================================================
def gen_twosin(w, R, N):
# Generates R random realizations of two sinusoids in random noise
#
# -- Output
# x[R, N] : R process realizations of N samples 
# 
# -- Input
# w[2] : frequency of sinusoids (rad/sample): [w1, w2]
# R    : number of realizations
# N    : number of samples

    print('===== Two sinusoids in noise ==========')
    
    # generate random realizations
    x = np.zeros((R, N))
    n = np.arange(N)

    for i in range(R):
        np.random.seed(i)
        x[i, :] = 10*np.sin(w[0]*n + 2*pi*np.random.rand()) + 5*np.sin(w[1]*n + 2*pi*np.random.rand()) + np.random.randn(1, N)
        
    return x

# ======================================================================
def gen_ma2(R, N):
# Generates R random realizations of MA(2) process
#
# -- Output
# x[R, N] : R process realizations of N samples 
# 
# -- Input
# R    : number of realizations
# N    : number of samples

    print('===== MA(2) process ==========')
    
    # generate random realizations
    x = np.random.randn(R, N+2)   # x will actually be of size (R, N); add two columns to account for the
                                  # two-sample delay
    x = x[:, 2:] - x[:, :-2]      # x[n] = e[n] - e[n-2]
    
    return x

# ======================================================================
def plot_realization(x, info):
# Plots one signal realization
#
# x[R, N] : R realizations of N-sample random process
# info    : a string with information to show in plot title
  
    [R, N] = x.shape
    
    # plot one signal realization
    xabsmax = np.max(abs(x))
    fig = plt.figure(figsize = (10, 4))
    fig.suptitle(info)
    plt.stem(range(N), x[1,:])
    plt.xlabel(r'$n$')
    plt.ylabel(r'$x(n)$') 
    plt.axis([-1, N, -1.1*xabsmax, 1.1*xabsmax])
    plt.grid()
    
    return

# ======================================================================
def acs_estimates(x, max_lag):
# Computes biased autocorrelation sequence (ACS) estimates for input
# signal realization at given lags
#
# -- Output
# r[R, max_lag+1] : biased ACS estimats at lags 0 to max_lag for each of 
#                   the R input signal realizations
#
# -- Input
# x[R, N]      : input signal (R realizations; N samples per realization) 
# max_lag      : maximum time lag for which the ACS is to be estimated

    [R, N] = x.shape
    r = np.zeros((R, max_lag + 1))
    
    for k in range(max_lag + 1):
        c = np.zeros((R))
        for n in range(k, N):
            c += x[:, n]*np.conjugate(x[:, n - k])  # elementwise product in Python
              
        r[:, k] = c    
    
    r /= N   # biased ACS estimate -> use 'N - k' for unbiased estimate
    
    return r

# ======================================================================
def ar(x, p, Nfft, verbose = True):
# Computes AR(p) spectral estimate of random process realizations
#
# -- Output
# Sx[R, Nfft] : R realizations of spectral estimates over Nfft frequency points in [0, 2*pi] rad/sample
# r[R, p+1]   : R realizations of ACS estimates, from lag 0 to p
# sg2[R]      : innovation variance (modeling error) for each realization
# 
# -- Input
# x[R, N] : R realizations of N-sample random process
# p       : AR model order
# Nfft    : number of FFT points
# verbose : if true, verbose operation
      
    [R, N] = x.shape
    
    Sx = np.zeros((R, Nfft))          # AR(p) spectral estimate for each realization
    sg2 = np.zeros((R, 1))            # modeling error
    w = np.arange(0, 2*pi, 2*pi/Nfft) # frequency axis

    print('>>> AR(' + str(p) + ') spectral estimate with N = ' + str(N) + ' samples')
    
    # compute and plot ACS estimates
    r = acs_estimates(x, p)   # max lag = AR model order
    
    if verbose:
        info = 'ACS estimate of process X(n), N = ' + str(N) + ' samples'
        plot_realization(r, info)
  
    for i in range(R):
            # build autocorrelation matrix
            Rx = toeplitz(r[i, :p])
            
            # build independent vector
            rp = r[i, 1:p+1]
            
            # solve Yule-Walker equations
            a = np.linalg.solve(Rx, -rp)
            
            # compute modeling error (innovation variance)
            sg2[i] = r[i, 0] + np.dot(np.conjugate(rp), a)
            
            # compute parametric PSD estimate at given frequency points
            expjw = np.exp(-1j*np.ma.outerproduct(np.arange(1, p+1), w)); # expjw[k, m] = exp(-j*(k+1)*m/Nfft)          
            Sx[i, :] = sg2[i]/abs(1 + np.dot(a, expjw))**2
     
    if verbose:
        print('- average modeling error = ', round(np.mean(sg2), 4))
  
    return Sx, r, sg2
    
# ======================================================================
def periodogram(x, Nfft = 1024): 
# Computes periodogram of random process realizations
#
# -- Output
# Sx[R, Nfft] : R realizations of spectral estimates over Nfft frequency points in [0, 2*pi] rad/sample
#
# -- Input
# x[R, N] : R realizations of N-sample random process
# Nfft    : number of FFT points
    
    [R, N] = x.shape
    
    print('>>> Periodogram with N = ' + str(N) + ' samples')
  
    # compute FFT of each realization
    X = fft(x, Nfft, 1)     # note: axes start counting from 0

    # compute periodogram
    Sx = abs(X)**2/N
    
    return Sx
    
# ======================================================================
# ======================================================================
def blackman_tuckey(r, window, Nfft):  # < TO BE COMPLETED >
# Computes Blackman-Tuckey's spectral estimate of random process realizations
#
# -- Output
# Sx[R, Nfft] : R realizations of spectral estimates over Nfft frequency points in [0, 2*pi] rad/sample
#
# -- Input
# r[R, N] : R realizations of autocorrelation sequence at lags 0 to N-1 
# window  : lag window; size determines BT truncation length of the ACS: lags -M+1 to M-1
# Nfft    : number of FFT points
    
    R = r.shape[0]
   
    M = int((window.shape[1] + 1)/2)
    
    Sx = np.zeros((R, Nfft))     # BT's spectral estimate for each realization
    
    for i in range(R):
        # compute Blackman-Tuckey's estimate
        Sx[i, :] = abs(fft(window*np.concatenate((r[i, M-1:0:-1], r[i, :M])), Nfft))  # use 'abs' to avoid potential residual imaginary values
                       
    return Sx

# ======================================================================

def spectral_bias_variance(Sx, Sx_bounds, info):
# Computes and plots bias and variance of spectral estimate
#
# -- Output
# w  : frequency points (rad/sample) where PSD is plotted
#
# -- Input
# Sx[R, Nfft] : R realizations of spectral estimates over Nfft frequency points in [0, 2*pi] rad/sample
# Sx_bounds = [realization_min, realization_max, average_min, average_max]: bounds for spectral representation (dB) 
# info : a string with additional information to appear in plots (number of segments, overlap, etc.)

    [R, Nfft] = Sx.shape
  
    # plot PSD realizations
    w = np.arange(Nfft/2)*2/Nfft  # sampled frequency axis (relative to pi)
    fig = plt.figure(figsize = (12, 4))
    fig.suptitle('Spectral estimate realizations, mean and variance of random process X(n) - ' + info)
    plt.subplot(1, 2, 1)
    
    for i in range(R):
        plt.plot(w, 10*np.log10(Sx[i, :int(Nfft/2)])) 
    
    plt.xlabel(r'$\omega/\pi$')
    plt.ylabel(r'$S_X(\omega)\ (dB)$')
    plt.axis([0, 1, Sx_bounds[0], Sx_bounds[1]])
    plt.grid()

    # compute average periodogram and variance
    Sx_ave = np.mean(Sx, 0)
    Sx_var = np.var(Sx, 0)
    plt.subplot(1, 2, 2)
    plt.plot(w, 10*np.log10(Sx_ave[:int(Nfft/2)]))   
    plt.plot(w, 10*np.log10(Sx_var[:int(Nfft/2)]))   
    plt.xlabel(r'$\omega/\pi$')
    plt.ylabel(r'$S_X(\omega)\ (dB)$')
    plt.legend(('mean', 'variance'))
    plt.axis([0, 1, Sx_bounds[2], Sx_bounds[3]])
    plt.grid()

    # average variance over all frequencies
    print('- average variance over all frequencies =',
                  round(10*np.log10(np.mean(Sx_var)), 2), 'dB')
    print()
    
    return pi*w

# ======================================================================


N = 64       # total length of signal
R = 50       # number of realizations
w1 = 0.4*pi
w2 = 0.6*pi
max_lag = 50
p = 4        # AR model order
Nfft = 1024  # FFT length
Sx_bounds = [-20, 50, -20, 50] # bounds for spectral representation (dB) [realization_min, realization_max, average_min, average_max]


x = gen_twosin([w1, w2], R, N)

# plot one signal realization
info = 'A realization of two sinusoids in noise random process X(n), N = ' + str(N) + ' samples'
plot_realization(x, info)

# compute AR spectral estimates
[Sx, r, sg2] = ar(x, p, Nfft)

# compute and plot bias and variance
info = 'AR(' + str(p) + ') model, N = ' + str(N) + ' samples'
spectral_bias_variance(Sx, Sx_bounds, info)

# repeat using the periodogram
#Sx = periodogram(x, Nfft)
    
# compute and plot bias and variance
info = 'Periodogram, N = ' + str(N) + ' samples'
spectral_bias_variance(Sx, Sx_bounds, info)


# compute Blackman-Tuckey's spectral estimates  < TO BE COMPLETED >
#window = np.ones((1, 2*p+1))   # rectangular window
#Sx = blackman_tuckey(r, window, Nfft)

# compute and plot bias and variance
#info = "Blackman-Tuckey, "+ str(2*p+1) + "-point rectangular window"
#spectral_bias_variance(Sx, Sx_bounds, info)


x = gen_ma2(R, N) # generate random realizations of MA(2) process

# plot one signal realization
info = 'A realization of MA(2) random process X(n), N = ' + str(N) + ' samples'
plot_realization(x, info)

# compute AR spectral estimates
[Sx, r, sg2] = ar(x, p, Nfft)

# compute and plot bias and variance
Sx_bounds = [-20, 20, -20, 20]
info = 'AR(' + str(p) + ') model, N = ' + str(N) + ' samples'
w = spectral_bias_variance(Sx, Sx_bounds, info)

# add actual PSD
Sx_true = 2 - 2*np.cos(2*w)
plt.plot(w/pi, 10*np.log10(Sx_true))  
plt.legend(('mean', 'variance', 'true PSD'))
  
# repeat using the periodogram
Sx = periodogram(x, Nfft)
    
# compute and plot bias and variance
info = 'Periodogram, N = ' + str(N) + ' samples'
spectral_bias_variance(Sx, Sx_bounds, info)

# add actual PSD
plt.plot(w/pi, 10*np.log10(Sx_true))  
plt.legend(('mean', 'variance', 'true PSD'))

## Spectral line splitting

**[HAY96, p. 443]** An artifact that may be observed with the autocorrelation method is **spectral line splitting**. This artifact consists of the splitting of a single spectral peak into two (or more) well separate and distinct peaks. Typically, this phenomenon occurs when $X(n)$ is overmodeled, i.e., when the assumed value of $p$ is too large as compared with the actual value of $p$ of the underlying model.


To illustrate this phenomenon, consider the **AR(2) process** given by the difference equation:

$$
X(n) = -0.9X(n - 2) + \varepsilon(n)
$$

where $\varepsilon(n)$ is a zero-mean unit-variance white noise (innovation process).


* Generate 5 independent random realizations of $N = 64$ samples of this AR(2) process. 


* Repeat the experiment of the previous section assuming an AR(4) model, and then an AR(12) model. What can be observed?



In [ ]:
# ======================================================================
def gen_ar2(R, N):
# Generates R random realizations of the AR(2) process
#
# -- Output
# x[R, N] : R process realizations of N samples 
# 
# -- Input
# R : number of realizations
# N : number of samples

    print('===== AR(2) process ==========')
    
    # generate random realizations
    e = np.random.randn(R, N+2)
    x = np.zeros((R, N+2)) # x will actually be of size (R, N); 
                           # we add two columns to account for the two-sample delay

    for n in range(2, N+2):
        x[:, n] = -0.9*x[:, n-2] + e[:, n]
    
    x = x[:, 2:N+2]  # avoid initial transient interval
    
    return x

# ======================================================================


R = 5   # number of realizations
Sx_bounds = [-20, 50, -20, 50]

x = gen_ar2(R, N) # generate random realizations of AR(2) process

# plot one signal realization
info = 'A realization of AR(2) random process X(n), N = ' + str(N) + ' samples'
plot_realization(x, info)

for p in [4, 12]:

    # compute AR spectral estimates
    [Sx, r, sg2] = ar(x, p, Nfft)
   
    # compute and plot bias and variance
    info = 'AR(' + str(p) + ') model, N = ' + str(N) + ' samples'
    spectral_bias_variance(Sx, Sx_bounds, info)



## Model order selection

**Akaike's information criterion (AIC)**, the **minimum description length (MDL)** criterion and Akaike's **final prediction error (FPE)** criterion are given, respectively, by:

$$\mathrm{AIC}(p) = N\log \sigma^2_\varepsilon + 2p$$

$$\mathrm{MDL}(p) = N\log \sigma^2_\varepsilon + (\log N)p$$

$$\mathrm{FPE}(p) = \sigma^2_\varepsilon \frac{N + p + 1}{N - p - 1}.$$



* Consider the **AR(2) model** of the previous exercise. For the realizations of $N = 64$ samples, compute and plot the AIC, MDL and FPE criteria as a function of the assumed model order $p$ in the range $[1, 10]$. Estimate the modeling error as the average innovation variance over the available signal realizations. Compare the different model order selection methods. Do they confirm that $p = 2$ is the right order for this AR process?


* Repeat for the **two sinusoids in additive noise** studied at the beginning of this notebook. Do the order selection criteria confirm that AR(4) is a suitable model for this process?


* What about the selected order for the **MA(2) process**?


* Justify the shape of the curves for the different criteria in each case.


In [ ]:
# ======================================================================
def model_order(x, pmax, info):
# Determines and plots model order selection criteria (AIC, MDL, FPE)
#
# -- Input
# x[R, N] : input signal realizations
# pmax    : maximum model order
# info    : string with additional information for plots

    sg2 = np.zeros(pmax)
    
    for p in range(1, pmax+1):
        [Sx, r, sg2_current] = ar(x, p, 2, False) # just 2 FFT points suffice, since the PSD is not actually needed
        sg2[p-1] = np.mean(sg2_current)  # average modeling error over realizations
    
    # compute model order selection criteria
    C = np.zeros((3, pmax))
    C[0, :] = N*np.log10(sg2) + 2*p 
    C[1, :] = N*np.log10(sg2) + np.log10(N)*p
    C[2, :] = sg2*(N + p + 1)/(N - p - 1)

    # plot criteria vs model order
    fig = plt.figure(figsize = (10, 4))
    fig.suptitle('Model order selection criteria - ' + info)
    p = range(1, pmax+1)
    
    for i in range(3):
        plt.plot(p, C[i,:])
    
    plt.xlabel('$p$')
    plt.ylabel('C(p)') 
    plt.grid()
    plt.legend(('AIC', 'MDL', 'FPE'))    

    return

# ======================================================================


pmax = 10

x = gen_ar2(R, N)
info = 'AR(2) process, N = ' + str(N) + ' samples'
model_order(x, pmax, info)

x = gen_twosin([w1, w2], R, N)
info = 'two sinusoids in additive noise, N = ' + str(N) + ' samples'
model_order(x, pmax, info)

x = gen_ma2(R, N)
info = 'MA(2) process, N = ' + str(N) + ' samples'
model_order(x, pmax, info)
